# Collaborative Filtering

Collaborative Filtering means that interests of other users are taking into account for making a playlist prediction. Spotify uses a matrix where each row is a user and each column is a song, each entry therefore resembles how often a user has listened to a song. Since this data is not publicly available, we will try the same approach, but with a different matrix. The matrix will have one row for each playlist and each column will be a song. The entries of the matrix are going to be 1 if the song is in the playlist and 0 if not. Read more about collaborative filtering [here](TODO).

Let's first import all important libraries. pandas DataFrames are used to store the data and numpy arrays for matrix computations.

In [15]:
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm

from scripts.matrix_factorization import MF  # for matrix factorization

Load the raw data.

In [2]:
df = pd.read_csv("data/raw_data.csv")
print(f"We have a dataset of {len(df)} entries")

We have a dataset of 67503 entries


We want to build a matrix where each row is a playlist, and each column resembles a song. The dimensions of our matrix resemble those of the playlist number and track number.

In [3]:
num_playlists = df["name"].nunique()  # count distinct values, this is the number of playlists
num_tracks = df["track_name"].nunique()  # count distinct values, this is the number of tracks
print(f"Playlists: {num_playlists} \nTracks: {num_tracks}")

Playlists: 869 
Tracks: 30049


Group the data by the playlist name. This results in a Series.

In [4]:
    playlists = df.groupby('name')["track_name"].apply(list)
playlists.head()

name
 CHiLl         [Make Me (Cry), Party Monster, Don't Wanna Kno...
 Frozen        [Frozen Heart, Do You Want to Build a Snowman?...
 indie rock    [Be Good (RAC Remix), Bambi, Your English Is G...
#Relaxed       [All That I Can Say, Reminisce, Butterfly, Cha...
#Workout       [Can't Feel My Face - Martin Garrix Remix, Ign...
Name: track_name, dtype: object

We need a list of the unique songs to create each playlist vector.

In [13]:
unique_songs = list(df["track_name"].unique())
print(f"Number of unique songs: {len(unique_songs)}")

Number of unique songs: 30049


We can now iteratively build our matrix by creating each playlist in a vector of its songs. This is done by one-hot encoding, meaning every column is a song, and every row a playlist, and the row-column combination is one if the song was added to the playlist. 

In [6]:
one_hot_playlists = list()
for playlist in playlists:
    playlist_array = np.zeros(num_tracks)
    for song in playlist:
        playlist_array[unique_songs.index(song)] = 1  # set array to 1 at index of the song
    one_hot_playlists.append(playlist_array)
one_hot_playlists = np.array(one_hot_playlists)  # convert to one numpy array (matrix)

For example, the first playlist "CHiLl" includes the song "Make Me (Cry)" and does not include "Mr. Brightside". Check if the value is one and zero respectively at the corresponding positions.

In [7]:
print(one_hot_playlists[0][unique_songs.index("Make Me (Cry)")] == 1.0)
print(one_hot_playlists[0][unique_songs.index("Mr. Brightside")] == 0.0)

True
True


The shape of our playlists should be playlist number times distinct track number.

In [8]:
one_hot_playlists.shape

(869, 30049)

We now apply matrix factorization on our data. This means, we try to find two matrices, which multiplied are as close to the original matrix as possible. We train using gradient descent, meaning we try to minimize the error in each iteration.

In [10]:
mf = MF(one_hot_playlists, K=2, alpha=0.1, beta=0.01, iterations=100)
mf.train()

Iteration: 10 ; error = 2.1933
Iteration: 20 ; error = 1.1882
Iteration: 30 ; error = 0.7869
Iteration: 40 ; error = 0.5708
Iteration: 50 ; error = 0.4376
Iteration: 60 ; error = 0.3490
Iteration: 70 ; error = 0.2851
Iteration: 80 ; error = 0.2394
Iteration: 90 ; error = 0.2034
Iteration: 100 ; error = 0.1756


[(0, 15.169171582728703),
 (1, 8.436861549375065),
 (2, 5.928733198897154),
 (3, 4.618687998363574),
 (4, 3.840796404704238),
 (5, 3.298640036942886),
 (6, 2.9205146740147545),
 (7, 2.6083174687270576),
 (8, 2.370648468564867),
 (9, 2.193254449675297),
 (10, 2.011621812287397),
 (11, 1.8653862665428302),
 (12, 1.7490904693645717),
 (13, 1.6359577290150913),
 (14, 1.542211557321664),
 (15, 1.4525396757193663),
 (16, 1.3860796270519946),
 (17, 1.3053195552218728),
 (18, 1.2470859696591141),
 (19, 1.1881813523809308),
 (20, 1.1267613455694327),
 (21, 1.0786249879839842),
 (22, 1.0301422453595313),
 (23, 0.9917154623291724),
 (24, 0.9507833855453965),
 (25, 0.9124269911379166),
 (26, 0.8774356896802425),
 (27, 0.8452000859598309),
 (28, 0.8134168358983895),
 (29, 0.7868838548031866),
 (30, 0.7622983149255489),
 (31, 0.7323430956886005),
 (32, 0.708089943487124),
 (33, 0.685645216908021),
 (34, 0.6636301226149542),
 (35, 0.6427883373542986),
 (36, 0.6237155899630551),
 (37, 0.60414527842555

As a result, we get a matrix where values should be close to their original values, but unknown values, in our case songs that are not in the playlist, are approximated by the matrix factorization. For example, if we look at the same song from earlier, the value is close to 1. Looking in at a song that has not been added to the playlist, the value is now approximated to how likely it should be added to the playlist.

In [11]:
print(mf.full_matrix()[0][unique_songs.index("Make Me (Cry)")])
print(mf.full_matrix()[0][unique_songs.index("Mr. Brightside")])

0.999370629663436
1.0001387417379277


We use this matrix for making our recommendation. We do so by looking for the songs with the least difference to the number 1, since this number was given to songs that were in the playlist. Let's predict 5 more songs for our first playlist "ChiLl".

In [45]:
rec_matrix = mf.full_matrix()
playlist = 0  # choose which playlist to look at
playlist_vals = rec_matrix[playlist]  # all predicted values for the first playlist
diff_vals = playlist_vals - 1  # subtract 1 from each value to get difference to 1

k=5  # find the k smallest values
smallest_k_indexes = np.argpartition(diff_vals, k)  # get list of indexes ordered according to descending order of values

i = 0  # record how many songs were output
for idx in np.flip(smallest_k_indexes):  # look at which song corresponds to found indexes
    if one_hot_playlists[playlist][idx] == 0:  # song was not in playlist already
        print(unique_songs[idx])
        i += 1
    if i == k:
        break

Cosmic Angel - Acoustic From Capitol Studios
Crazy In Love
The Answer
Rock Your Body
It Wasn't Me
